In [4]:
import torch
import torch.nn.functional as F

def TV(img, gt, iter, dt, epsilon):
    batch, NX, NY = img.shape
    ep2 = epsilon * epsilon

    I_t = img.clone().float()
    I_tmp = img.clone().float()
    data = []
    for t in range(iter):
        # 计算梯度
        I_t_x = F.pad(I_t[:, :, 1:] - I_t[:, :, :-1], (0, 1), 'constant', 0)
        I_t_y = F.pad(I_t[:, 1:, :] - I_t[:, :-1, :], (0, 0, 0, 1), 'constant', 0)
        
        # 计算二阶导数
        I_t_xx = F.pad(I_t_x[:, :, 1:] - I_t_x[:, :, :-1], (0, 1), 'constant', 0)
        I_t_yy = F.pad(I_t_y[:, 1:, :] - I_t_y[:, :-1, :], (0, 0, 0, 1), 'constant', 0)
        I_t_xy = F.pad(I_t_x[:, 1:, :] - I_t_x[:, :-1, :], (0, 0, 0, 1), 'constant', 0)
        
        # 计算数值和分母
        tmp_x2 = I_t_x ** 2
        tmp_y2 = I_t_y ** 2
        tmp_num = I_t_yy * (tmp_x2 + ep2) + I_t_xx * (tmp_y2 + ep2) - 2 * I_t_x * I_t_y * I_t_xy
        tmp_den = (tmp_x2 + tmp_y2 + ep2) ** 1.5
        
        # 更新 I_tmp
        I_tmp += dt * (tmp_num / tmp_den + 0.5 * (img - I_t))
        
        # 更新 I_t
        I_t = I_tmp.clone()

        # 计算损失
        loss = ((I_t - gt) ** 2).mean().item()
        if t % 10 == 0:
            print(f'Iteration {t}, Loss: {loss}')
            # data.append(loss)
    
    # data = torch.tensor(data)
    return I_t

# 加载图像
decoded = torch.load('decoded2.pt').cpu()
gt = torch.load('gt2.pt').cpu()
print(f'img:{decoded.shape},gt:{gt.shape}')
img = torch.Tensor(decoded)  # 保留原始形状
TVimg = TV(decoded, gt, iter=100, dt=0.1, epsilon=1e-3)

# 计算损失
mseloss = torch.nn.L1Loss(reduction='sum')
loss0 = mseloss(img, gt)
loss4 = mseloss(TVimg, gt)
print(f'Mseloss原来loss={loss0:.4f}，TV后loss={loss4:.4f}')

img:torch.Size([2, 361, 720]),gt:torch.Size([2, 361, 720])
Iteration 0, Loss: 0.31067517399787903
Iteration 10, Loss: 1.5072685480117798
Iteration 20, Loss: 2.082674503326416
Iteration 30, Loss: 2.916780948638916
Iteration 40, Loss: 4.372099876403809
Iteration 50, Loss: 3.881906032562256
Iteration 60, Loss: 4.879528999328613
Iteration 70, Loss: 4.230108261108398
Iteration 80, Loss: 4.0122575759887695
Iteration 90, Loss: 4.754225254058838
Mseloss原来loss=204151.9062，TV后loss=777076.8750


In [5]:
import torch
import torch.nn.functional as F

def TV(img, gt, iter, dt, epsilon):
    batch, NX, NY = img.shape
    ep2 = epsilon * epsilon

    I_t = img.clone().float()
    I_tmp = img.clone().float()
    data = []
    for t in range(iter):
        # 计算梯度
        I_t_x = F.pad(I_t[:, :, 1:] - I_t[:, :, :-1], (0, 1), 'constant', 0)
        I_t_y = F.pad(I_t[:, 1:, :] - I_t[:, :-1, :], (0, 0, 0, 1), 'constant', 0)
        
        # 计算二阶导数
        I_t_xx = F.pad(I_t_x[:, :, 1:] - I_t_x[:, :, :-1], (0, 1), 'constant', 0)
        I_t_yy = F.pad(I_t_y[:, 1:, :] - I_t_y[:, :-1, :], (0, 0, 0, 1), 'constant', 0)
        I_t_xy = F.pad(I_t_x[:, 1:, :] - I_t_x[:, :-1, :], (0, 0, 0, 1), 'constant', 0)
        
        # 计算数值和分母
        tmp_x2 = I_t_x ** 2
        tmp_y2 = I_t_y ** 2
        tmp_num = I_t_yy * (tmp_x2 + ep2) + I_t_xx * (tmp_y2 + ep2) - 2 * I_t_x * I_t_y * I_t_xy
        tmp_den = (tmp_x2 + tmp_y2 + ep2) ** 1.5
        
        # 更新 I_tmp
        I_tmp += dt * (tmp_num / tmp_den)
        
        # 更新 I_t
        I_t = I_tmp.clone()

        # 计算损失
        loss = ((I_t - gt) ** 2).mean().item()
        if t % 10 == 0:
            print(f'Iteration {t}, Loss: {loss}')
            data.append(loss)
    
    # data = torch.tensor(data)
    return I_t

# 加载图像
decoded = torch.load('decoded2.pt').cpu()
gt = torch.load('gt2.pt').cpu()
print(f'img:{decoded.shape},gt:{gt.shape}')
img = torch.Tensor(decoded)  # 保留原始形状
TVimg = TV(decoded, gt, iter=100, dt=0.01, epsilon=1e-3)

# 计算损失
mseloss = torch.nn.L1Loss(reduction='sum')
loss0 = mseloss(img, gt)
loss4 = mseloss(TVimg, gt)
print(f'Mseloss原来loss={loss0:.4f}，TV后loss={loss4:.4f}')

img:torch.Size([2, 361, 720]),gt:torch.Size([2, 361, 720])
Iteration 0, Loss: 0.27516672015190125
Iteration 10, Loss: 0.2842631936073303
Iteration 20, Loss: 0.3032064735889435
Iteration 30, Loss: 0.33425289392471313
Iteration 40, Loss: 0.3781004250049591
Iteration 50, Loss: 0.4353066086769104
Iteration 60, Loss: 0.5064877271652222
Iteration 70, Loss: 0.5918822288513184
Iteration 80, Loss: 0.6916010975837708
Iteration 90, Loss: 0.8059101104736328
Mseloss原来loss=204151.9062，TV后loss=388789.6875


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
# import matplotlib.pyplot as plt
import numpy as np

import torch
import torch.nn.functional as F

def TV(img, gt, iter, dt, epsilon):
    NX, NY = img.shape
    ep2 = epsilon * epsilon

    I_t = img.clone().float()
    I_tmp = img.clone().float()

    data = []

    for t in range(iter):
        # 计算梯度
        I_t_x = F.pad(I_t[:, 1:] - I_t[:, :-1], (0, 1), 'constant', 0)
        I_t_y = F.pad(I_t[1:, :] - I_t[:-1, :], (0, 0, 0, 1), 'constant', 0)

        # 计算二阶导数
        I_t_xx = F.pad(I_t_x[:, 1:] - I_t_x[:, :-1], (0, 1), 'constant', 0)
        I_t_yy = F.pad(I_t_y[1:, :] - I_t_y[:-1, :], (0, 0, 0, 1), 'constant', 0)
        I_t_xy = F.pad(I_t_x[1:, :] - I_t_x[:-1, :], (0, 0, 0, 1), 'constant', 0)

        # 计算数值和分母
        tmp_x2 = I_t_x ** 2
        tmp_y2 = I_t_y ** 2
        tmp_num = I_t_yy * (tmp_x2 + ep2) + I_t_xx * (tmp_y2 + ep2) - 2 * I_t_x * I_t_y * I_t_xy
        tmp_den = (tmp_x2 + tmp_y2 + ep2) ** 1.5

        # 更新 I_tmp
        I_tmp += dt * (tmp_num / tmp_den + 0.5 * (img - I_t))

        # 更新 I_t
        I_t = I_tmp.clone()

        # 计算损失
        loss = ((I_t - gt) ** 2).mean().item()
        if t % 10 == 0:
            print(f'Iteration {t}, Loss: {loss}')
            # data.append(loss)

    # data = torch.tensor(data)
    return I_t

class SmoothLoss(nn.Module):
    def __init__(self, alpha=1.0, beta=1.0):
        super(SmoothLoss, self).__init__()
        self.alpha = alpha
        self.beta = beta

    def forward(self, decoded, GT):
        # Calculate the MSE loss
        mse_loss = nn.MSELoss(reduction='sum')
        loss_mse = mse_loss(decoded, GT)

        diff1 = torch.abs(decoded[:,  :-1, :] - decoded[:,  1:, :])
        diff2 = torch.abs(decoded[:,  :, :-1] - decoded[:,  :, 1:])
        smoothness_loss = torch.sum(diff1) + torch.sum(diff2) * self.alpha
        # print(f"smoothloss:{smoothness_loss*self.beta:.4f},mseloss:{loss_mse:.4f}")
        total_loss = loss_mse + smoothness_loss * self.beta
        return total_loss

def median_filter2d(img, kernel_size=5):
    pad_size = kernel_size // 2    # 计算 padding 大小
    img_padded = F.pad(img, (pad_size, pad_size, pad_size, pad_size), mode='reflect')    # 对图像进行 padding
    batch_size, channels, height, width = img_padded.shape    # 获取图像的尺寸
    unfolded = F.unfold(img_padded, kernel_size=kernel_size)    # 展开图像矩阵
    unfolded = unfolded.view(batch_size, channels, kernel_size * kernel_size, -1)    # 计算中值
    median = unfolded.median(dim=2)[0]
    median = median.view(batch_size, channels, height - 2 * pad_size, width - 2 * pad_size)    # 恢复图像尺寸
    return median

def gaussian_kernel(size, sigma):
    """生成一个高斯核"""
    kernel = torch.tensor([[(1/(2.0*np.pi*sigma**2)) * np.exp(-((x - size//2)**2 + (y - size//2)**2)/(2*sigma**2))
                            for x in range(size)] for y in range(size)]).float()
    kernel /= kernel.sum()
    return kernel.unsqueeze(0).unsqueeze(0)

def gaussian_filter2d(img, kernel_size=5, sigma=1.0):
    """应用高斯滤波"""
    kernel = gaussian_kernel(kernel_size, sigma)
    channels = img.shape[1]
    kernel = kernel.repeat(channels, 1, 1, 1)
    padding = kernel_size // 2
    filtered_img = F.conv2d(img, kernel, padding=padding, groups=channels)
    return filtered_img

# 加载图像
decoded = torch.load('decoded2.pt').cpu()
gt = torch.load('gt2.pt').cpu()

sigma = 4.
# 假设图像是灰度图，转换形状为 [batch_size, channels, height, width]
# 这里的 decoded 是 [2, 361, 720]，我们假设 batch_size = 2，channels = 1
img = torch.Tensor(decoded).unsqueeze(1)  # 添加 channel 维度
filtered_img_median = median_filter2d(img, kernel_size=5)# 应用中值滤波
filtered_img_gaussian = gaussian_filter2d(img, kernel_size=5, sigma=sigma)# 应用高斯滤波
bothMG_img = gaussian_filter2d(filtered_img_median, kernel_size=5, sigma=sigma)#两个都用 这个效果好
bothGM_img = median_filter2d(filtered_img_median, kernel_size=5)#两个都用
TVimg = TV(decoded, gt, iter=100, dt=0.1, epsilon=1e-3)


# 移除 batch 和 channel 维度
img = img.squeeze(1)
filtered_img_median = filtered_img_median.squeeze(1)
filtered_img_gaussian = filtered_img_gaussian.squeeze(1)
bothMG_img = bothMG_img.squeeze(1)
bothGM_img = bothGM_img.squeeze(1)
gt = torch.Tensor(gt).squeeze(1)  # 确保 gt 形状正确


mseloss = torch.nn.MSELoss(reduction='sum')
loss0 = mseloss(img, gt)
loss1 = mseloss(filtered_img_median, gt)
loss2 = mseloss(filtered_img_gaussian, gt)
loss3 = mseloss(bothMG_img,gt)
loss4 = mseloss(bothGM_img,gt)
print(f'Mseloss原来loss={loss0:.4f}，中值滤波后loss={loss1:.4f}，高斯滤波后loss={loss2:.4f}，先中值后高斯loss={loss3:.4f}，两次中值后loss={loss4:.4f}')
smloss = SmoothLoss()
smloss0 = smloss(img, gt)
smloss1 = smloss(filtered_img_median, gt)
smloss2 = smloss(filtered_img_gaussian, gt)
smloss3 = smloss(bothMG_img,gt)
smloss4 = smloss(bothGM_img,gt)
print(f'Smoothloss原来loss={smloss0:.4f}，中值滤波后loss={smloss1:.4f}，高斯滤波后loss={smloss2:.4f}，先中值后高斯loss={smloss3:.4f}，两次中值后loss={smloss4:.4f}')

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import math

def TV(img, gt, iter, dt, epsilon, lamb):
    NX = img.shape[0]
    NY = img.shape[1]
    ep2 = epsilon * epsilon
    I_t=np.array(np.zeros((NX,NY)))
    I_tmp = np.array(np.ones((NX,NY)))  # 用来迭代的噪声图
    I_t = img.astype(np.float64)
    I_tmp = img.astype(np.float64)
    data = [] 
    for t in range(0, iter):
        for i in range(0, NY):  # 一次迭代
            for j in range(0, NX):
                iUp = i - 1
                iDown = i + 1
                jLeft = j - 1
                jRight = j + 1  # 边界处理
                if i == 0:
                    iUp = i
                if NY - 1 == i:
                    iDown = i
                if j == 0:
                    jLeft = j
                if NX - 1 == j:
                    jRight = j
                tmp_x = (I_t[i][jRight] - I_t[i][jLeft]) / 2.0
                tmp_y = (I_t[iDown][j] - I_t[iUp][j]) / 2.0
                tmp_xx = I_t[i][jRight] + I_t[i][jLeft] - 2 * I_t[i][j]
                tmp_yy = I_t[iDown][j] + I_t[iUp][j] - 2 * I_t[i][j]
                tmp_xy = (I_t[iDown][jRight] + I_t[iUp][jLeft] - I_t[iUp][jRight] - I_t[iDown][jLeft]) / 4.0
                tmp_num = tmp_yy * (tmp_x * tmp_x + ep2) + tmp_xx * (tmp_y * tmp_y + ep2) - 2 * tmp_x * tmp_y * tmp_xy
                tmp_den = math.pow(tmp_x * tmp_x + tmp_y * tmp_y + ep2, 1.5)
                I_tmp[i][j] += dt * (tmp_num / tmp_den + (0.5+lamb[i][j])* (img[i][j] - I_t[i][j]))

        for i in range(0, NY):
            for j in range(0, NX):
                I_t[i][j] = I_tmp[i][j]  # 迭代结束
        loss=((I_t - gt) ** 2).mean()
        if(t%10==0):
           print(loss)
           data.append(loss) 
    data=np.array(data)     
    return I_t  # 返回去噪图

class SmoothLoss(nn.Module):
    def __init__(self, alpha=1.0, beta=1.0):
        super(SmoothLoss, self).__init__()
        self.alpha = alpha
        self.beta = beta

    def forward(self, decoded, GT):
        # Calculate the MSE loss
        mse_loss = nn.MSELoss(reduction='sum')
        loss_mse = mse_loss(decoded, GT)

        diff1 = torch.abs(decoded[:,  :-1, :] - decoded[:,  1:, :])
        diff2 = torch.abs(decoded[:,  :, :-1] - decoded[:,  :, 1:])
        smoothness_loss = torch.sum(diff1) + torch.sum(diff2) * self.alpha
        # print(f"smoothloss:{smoothness_loss*self.beta:.4f},mseloss:{loss_mse:.4f}")
        total_loss = loss_mse + smoothness_loss * self.beta
        return total_loss

def median_filter2d(img, kernel_size=5):
    pad_size = kernel_size // 2    # 计算 padding 大小
    img_padded = F.pad(img, (pad_size, pad_size, pad_size, pad_size), mode='reflect')    # 对图像进行 padding
    batch_size, channels, height, width = img_padded.shape    # 获取图像的尺寸
    unfolded = F.unfold(img_padded, kernel_size=kernel_size)    # 展开图像矩阵
    unfolded = unfolded.view(batch_size, channels, kernel_size * kernel_size, -1)    # 计算中值
    median = unfolded.median(dim=2)[0]
    median = median.view(batch_size, channels, height - 2 * pad_size, width - 2 * pad_size)    # 恢复图像尺寸
    return median

def gaussian_kernel(size, sigma):
    """生成一个高斯核"""
    kernel = torch.tensor([[(1/(2.0*np.pi*sigma**2)) * np.exp(-((x - size//2)**2 + (y - size//2)**2)/(2*sigma**2))
                            for x in range(size)] for y in range(size)]).float()
    kernel /= kernel.sum()
    return kernel.unsqueeze(0).unsqueeze(0)

def gaussian_filter2d(img, kernel_size=5, sigma=1.0):
    """应用高斯滤波"""
    kernel = gaussian_kernel(kernel_size, sigma)
    channels = img.shape[1]
    kernel = kernel.repeat(channels, 1, 1, 1)
    padding = kernel_size // 2
    filtered_img = F.conv2d(img, kernel, padding=padding, groups=channels)
    return filtered_img

# 加载图像
decoded = torch.load('decoded2.pt').cpu()
gt = torch.load('gt2.pt').cpu()

sigma = 4.
# 假设图像是灰度图，转换形状为 [batch_size, channels, height, width]
# 这里的 decoded 是 [2, 361, 720]，我们假设 batch_size = 2，channels = 1
img = torch.Tensor(decoded).unsqueeze(1)  # 添加 channel 维度
filtered_img_median = median_filter2d(img, kernel_size=5)# 应用中值滤波
filtered_img_gaussian = gaussian_filter2d(img, kernel_size=5, sigma=sigma)# 应用高斯滤波
bothMG_img = gaussian_filter2d(filtered_img_median, kernel_size=5, sigma=sigma)#两个都用 这个效果好
bothGM_img = median_filter2d(filtered_img_median, kernel_size=5)#两个都用

# 移除 batch 和 channel 维度
img = img.squeeze(1)
filtered_img_median = filtered_img_median.squeeze(1)
filtered_img_gaussian = filtered_img_gaussian.squeeze(1)
bothMG_img = bothMG_img.squeeze(1)
bothGM_img = bothGM_img.squeeze(1)
gt = torch.Tensor(gt).squeeze(1)  # 确保 gt 形状正确

plt.figure(figsize=(20, 10))
# 显示原图和处理后的图像
for i in range(img.shape[0]):
    plt.subplot(2, 6, 6 * i + 1)
    plt.imshow(img[i].cpu().detach().numpy(), cmap='jet')
    plt.title(f'Original Image {i+1}')
    
    plt.subplot(2, 6, 6 * i + 2)
    plt.imshow(filtered_img_median[i].cpu().detach().numpy(), cmap='jet')
    plt.title(f'Median Filtered Image {i+1}')

    plt.subplot(2, 6, 6 * i + 3)
    plt.imshow(filtered_img_gaussian[i].cpu().detach().numpy(), cmap='jet')
    plt.title(f'Gaussian Filtered Image {i+1}')

    plt.subplot(2, 6, 6 * i + 4)
    plt.imshow(bothMG_img[i].cpu().detach().numpy(), cmap='jet')
    plt.title(f'G+M Filtered Image {i+1}')

    plt.subplot(2, 6, 6 * i + 5)
    plt.imshow(bothGM_img[i].cpu().detach().numpy(), cmap='jet')
    plt.title(f'M+M Filtered Image {i+1}')
    
    plt.subplot(2, 6, 6 * i + 6)
    plt.imshow(gt[i].cpu().detach().numpy(), cmap='jet')
    plt.title(f'GT Image {i+1}')
# plt.figure(figsize=(20, 10))
plt.subplots_adjust(wspace=0.01, hspace=0.01)
plt.show()

mseloss = torch.nn.MSELoss(reduction='sum')
loss0 = mseloss(img, gt)
loss1 = mseloss(filtered_img_median, gt)
loss2 = mseloss(filtered_img_gaussian, gt)
loss3 = mseloss(bothMG_img,gt)
loss4 = mseloss(bothGM_img,gt)
print(f'Mseloss原来loss={loss0:.4f}，中值滤波后loss={loss1:.4f}，高斯滤波后loss={loss2:.4f}，先中值后高斯loss={loss3:.4f}，两次中值后loss={loss4:.4f}')
smloss = SmoothLoss()
smloss0 = smloss(img, gt)
smloss1 = smloss(filtered_img_median, gt)
smloss2 = smloss(filtered_img_gaussian, gt)
smloss3 = smloss(bothMG_img,gt)
smloss4 = smloss(bothGM_img,gt)
print(f'Smoothloss原来loss={smloss0:.4f}，中值滤波后loss={smloss1:.4f}，高斯滤波后loss={smloss2:.4f}，先中值后高斯loss={smloss3:.4f}，两次中值后loss={smloss4:.4f}')